<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-18 11:14:40
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  4.98 L
Current:  1.43 C
-------------------
Today PnL: -25.48 K (-0.18%)
Current PnL: -31.58 L (-19.74%)
CY Booked + Current PnL: -15.90 L (-9.94%)
-------------------
Total profit:  1.28 L
Total loss:  -32.86 L
-------------------
Total Booked + Current PnL: 11.08 L (8.33%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (10.98%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.40 C
Est FTT PnL: 96.92 L (67.84%)
Deployed:  1.33 C
Current:  1.43 C
CAGR/XIRR %: 3.71%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.44,3.89,17.02,21.57,17325.0,3810.0,101790.0,397.04,-10.57,61.0,H-SC,4.92,167.0,0.22,0.74,48.54,XR,ATH,FINANCE
18,COALINDIA,-0.26,2.04,15.32,17.67,23521.0,3072.0,153532.0,478.51,23.39,48.0,L-LC,11.33,185.0,0.13,1.11,23.25,XY25,ATH,MINING
77,TTKPRESTIG,-0.16,-27.43,37.92,0.09,27734.0,-27640.0,73137.0,770.00,69.69,39.0,M-SC,9.36,250.0,-1.00,0.53,0.66,OX40N,NTT,DURABLES
36,ICICIGI,0.64,2.79,16.10,19.34,33117.0,5584.0,205693.0,2252.93,-13.46,62.0,X-MC,2.87,66.0,0.17,1.49,19.17,X40,ATH,INSURANCE
51,MEDANTA,-0.41,-2.51,27.27,24.08,33751.0,-3184.0,123766.0,1486.00,-3.14,57.0,X-SC,11.04,87.0,-0.09,0.90,13.36,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,5.47,-44.70,163.47,45.70,159833.0,-79040.0,97775.0,26.40,1.63,83.0,M-SC,22.75,204.0,-0.49,0.71,61.09,XY24,NTT,TRAVEL
62,SATIN,2.27,-12.06,72.66,51.83,176530.0,-33330.0,242954.0,274.00,-21.16,59.0,H-SC,4.31,159.0,-0.19,1.76,18.35,XY24,NTT,FINANCE
57,RECLTD,1.58,-1.39,22.44,20.74,44956.0,-2832.0,200338.0,446.00,9.37,53.0,M-MC,3.00,191.0,-0.06,1.45,10.40,XY25,NTT,FINANCE
44,ITC,1.46,-19.07,36.89,10.78,71747.0,-45830.0,194488.0,452.00,-52.28,54.0,X-LC,18.68,1.0,-0.64,1.41,7.12,X40,NTT,FMCG
20,COLPAL,1.43,-17.16,70.81,41.51,154493.0,-45185.0,218180.0,3726.84,0.20,59.0,X-MC,6.23,62.0,-0.29,1.58,6.61,XY25,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,-2.88,-24.71,113.53,60.77,156229.0,-45165.0,137610.0,521.00,58.68,54.0,H-SC,1.42,160.0,-0.29,1.00,24.76,XR,NTT,JEWELLERY
23,DIXON,-2.44,-21.29,66.76,31.26,121268.0,-49120.0,181648.0,18931.72,-64.81,48.0,X-MC,25.71,54.0,-0.41,1.32,11.48,X40N,ATH,IT
14,BSOFT,-2.27,-31.63,120.55,50.79,115921.0,-44493.0,96160.0,831.70,-12.25,36.0,H-SC,7.01,172.0,-0.38,0.70,11.96,XR,ATH,IT
84,WIPRO,-1.87,-13.19,98.43,72.27,144794.0,-22344.0,147104.0,420.00,-21.74,26.0,M-LC,6.44,98.0,-0.15,1.07,0.00,XR,NTT,IT
0,5PAISA,-1.82,-38.79,84.45,12.90,112947.0,-84752.0,133744.0,593.00,97.12,40.0,H-SC,6.19,170.0,-0.75,0.97,11.21,OX40N,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,0.04,3.73,100.43,107.91,153812.0,5509.0,153153.0,1641.55,-6.65,65.0,M-SC,8.80,220.0,0.04,1.11,15.22,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.16,-27.43,37.92,0.09,27734.0,-27640.0,73137.0,770.00,69.69,39.0,M-SC,9.36,250.0,-1.00,0.53,0.66,OX40N,NTT,DURABLES
66,SIS,0.89,-28.23,68.96,21.26,54953.0,-31344.0,79688.0,528.00,1874.61,38.0,H-SC,4.22,168.0,-0.57,0.58,9.86,OX40N,NTT,MISC
72,TATAELXSI,-1.35,-30.07,88.79,32.03,81863.0,-39637.0,92198.0,9161.00,-12.47,35.0,H-SC,7.26,158.0,-0.48,0.67,3.27,OX40N,NTT,IT
40,INDIGOPNTS,-0.49,-30.87,44.68,0.02,53918.0,-53882.0,120677.0,1408.00,84.15,30.0,M-SC,8.68,222.0,-1.00,0.88,4.97,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.26,2.04,15.32,17.67,23521.0,3072.0,153532.0,478.51,23.39,48.0,L-LC,11.33,185.0,0.13,1.11,23.25,XY25,ATH,MINING
26,FINCABLES,0.04,3.73,100.43,107.91,153812.0,5509.0,153153.0,1641.55,-6.65,65.0,M-SC,8.80,220.0,0.04,1.11,15.22,OX40N,ATH,CABLES
50,MASFIN,0.44,3.89,17.02,21.57,17325.0,3810.0,101790.0,397.04,-10.57,61.0,H-SC,4.92,167.0,0.22,0.74,48.54,XR,ATH,FINANCE
1,ABB,-0.87,6.15,37.14,45.57,103139.0,16085.0,277704.0,7934.00,-31.79,65.0,H-MC,4.27,125.0,0.16,2.01,23.32,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.44,3.89,17.02,21.57,17325.0,3810.0,101790.0,397.04,-10.57,61.0,H-SC,4.92,167.0,0.22,0.74,48.54,XR,ATH,FINANCE
1,ABB,-0.87,6.15,37.14,45.57,103139.0,16085.0,277704.0,7934.00,-31.79,65.0,H-MC,4.27,125.0,0.16,2.01,23.32,AR,NTT,ELECTRICAL
26,FINCABLES,0.04,3.73,100.43,107.91,153812.0,5509.0,153153.0,1641.55,-6.65,65.0,M-SC,8.80,220.0,0.04,1.11,15.22,OX40N,ATH,CABLES
39,INDIAMART,-1.53,-5.90,119.69,106.72,138905.0,-7282.0,116054.0,4810.62,-57.62,45.0,H-SC,7.80,140.0,-0.05,0.84,16.09,AR,ATH,MISC
85,ZYDUSLIFE,-0.21,-4.60,42.17,35.63,84314.0,-9635.0,199939.0,1286.17,-17.12,51.0,H-MC,5.44,120.0,-0.11,1.45,11.31,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-1.82,-7.02,44.37,34.24,129089.0,-21951.0,290937.0,1972.00,-26.71,24.0,X-LC,3.94,3.0,-0.17,2.11,0.69,X40,NTT,IT
73,TCS,-1.16,-26.15,60.52,18.55,169059.0,-98893.0,279344.0,4311.58,-33.14,26.0,X-LC,4.22,2.0,-0.58,2.03,0.00,X40,ATH,IT
33,HCLTECH,-1.48,-5.21,30.64,23.83,70267.0,-12594.0,229330.0,1908.19,-0.33,27.0,X-LC,4.54,8.0,-0.18,1.66,10.03,X40,ATH,IT
8,AWL,-0.97,-34.51,140.02,57.19,299066.0,-112539.0,213588.0,485.00,-62.60,29.0,X-SC,17.57,81.0,-0.38,1.55,0.25,XY24,NTT,FMCG
13,BERGEPAINT,-0.65,-17.98,47.70,21.14,88942.0,-40877.0,186462.0,680.00,-19.82,30.0,X-MC,12.43,68.0,-0.46,1.35,0.41,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,-0.12,4.48,15.59,20.77,38139.0,10492.0,244636.0,1644.00,-7.40,63.0,X-MC,1.47,64.0,0.28,1.77,12.21,X40N,NTT,BREWERIES
21,DABUR,-1.54,-0.51,44.44,43.70,109674.0,-1276.0,246792.0,735.00,-9.68,47.0,X-MC,1.87,70.0,-0.01,1.79,14.46,XY24,BTT,FMCG
65,SIEMENS,0.15,-14.26,46.38,25.51,74006.0,-26530.0,159565.0,4671.50,33.99,56.0,H-LC,1.95,51.0,-0.36,1.16,17.59,AR,ATH,ELECTRICAL
34,HINDUNILVR,0.14,-9.92,26.19,13.67,60643.0,-25513.0,231550.0,2922.00,-33.13,41.0,X-LC,2.86,9.0,-0.42,1.68,9.31,XY25,NTT,FMCG
36,ICICIGI,0.64,2.79,16.10,19.34,33117.0,5584.0,205693.0,2252.93,-13.46,62.0,X-MC,2.87,66.0,0.17,1.49,19.17,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,-1.16,-26.15,60.52,18.55,169059.0,-98893.0,279344.0,4311.58,-33.14,26.0,X-LC,4.22,2.0,-0.58,2.03,0.00,X40,ATH,IT
53,PGHH,-0.03,-13.16,52.01,32.01,96439.0,-28096.0,185424.0,17616.87,-41.22,35.0,X-MC,9.32,57.0,-0.29,1.34,0.22,X40,ATH,FMCG
8,AWL,-0.97,-34.51,140.02,57.19,299066.0,-112539.0,213588.0,485.00,-62.60,29.0,X-SC,17.57,81.0,-0.38,1.55,0.25,XY24,NTT,FMCG
3,ACC,-0.05,-31.14,138.39,64.15,226752.0,-74101.0,163850.0,3906.00,-48.05,39.0,X-SC,8.48,85.0,-0.33,1.19,0.39,XY24,BTT,CEMENT
13,BERGEPAINT,-0.65,-17.98,47.70,21.14,88942.0,-40877.0,186462.0,680.00,-19.82,30.0,X-MC,12.43,68.0,-0.46,1.35,0.41,XY24,NTT,PAINTS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-0.27,-30.25,104.78,42.84,47511.0,-19662.0,45344.0,424.00,-54.27,51.0,X-SC,18.86,80.0,-0.41,0.33,9.07,XY24,NTT,MISC
12,BATAINDIA,-0.66,-46.25,151.53,35.20,104804.0,-59506.0,69164.0,2096.00,-9.31,36.0,X-SC,21.53,91.0,-0.57,0.50,0.54,X40,NTT,FOOTWEAR
58,RELAXO,-0.00,-52.32,219.44,52.31,151879.0,-75948.0,69212.0,1176.00,-49.28,44.0,X-SC,14.08,92.0,-0.50,0.50,3.76,X40N,NTT,FOOTWEAR
51,MEDANTA,-0.41,-2.51,27.27,24.08,33751.0,-3184.0,123766.0,1486.00,-3.14,57.0,X-SC,11.04,87.0,-0.09,0.90,13.36,XY24,NTT,HEALTHCARE
35,HONAUT,-0.13,-23.04,86.24,43.32,108093.0,-37532.0,125340.0,58357.33,-32.50,39.0,X-SC,11.16,90.0,-0.35,0.91,1.77,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,1.46,-19.07,36.89,10.78,71747.0,-45830.0,194488.0,452.00,-52.28,54.0,X-LC,18.68,1.0,-0.64,1.41,7.12,X40,NTT,FMCG
73,TCS,-1.16,-26.15,60.52,18.55,169059.0,-98893.0,279344.0,4311.58,-33.14,26.0,X-LC,4.22,2.0,-0.58,2.03,0.00,X40,ATH,IT
42,INFY,-1.82,-7.02,44.37,34.24,129089.0,-21951.0,290937.0,1972.00,-26.71,24.0,X-LC,3.94,3.0,-0.17,2.11,0.69,X40,NTT,IT
75,TMPV,-0.20,-21.68,57.03,22.98,100893.0,-48986.0,176912.0,600.00,-81.44,63.0,X-LC,2.95,4.0,-0.49,1.28,13.10,XY24,NTT,AUTO
33,HCLTECH,-1.48,-5.21,30.64,23.83,70267.0,-12594.0,229330.0,1908.19,-0.33,27.0,X-LC,4.54,8.0,-0.18,1.66,10.03,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.03,-30.34,90.77,32.89,48818.0,-23424.0,53782.0,1800.00,-353.50,64.0,L-MC,11.13,258.0,-0.48,0.39,48.11,XR,NTT,BANKS
67,SONACOMS,-0.07,-8.42,51.87,39.09,48055.0,-8514.0,92645.0,804.02,-27.49,63.0,M-MC,6.68,193.0,-0.18,0.67,31.06,AR,ATH,AUTO
25,EASEMYTRIP,5.47,-44.70,163.47,45.70,159833.0,-79040.0,97775.0,26.40,1.63,83.0,M-SC,22.75,204.0,-0.49,0.71,61.09,XY24,NTT,TRAVEL
50,MASFIN,0.44,3.89,17.02,21.57,17325.0,3810.0,101790.0,397.04,-10.57,61.0,H-SC,4.92,167.0,0.22,0.74,48.54,XR,ATH,FINANCE
18,COALINDIA,-0.26,2.04,15.32,17.67,23521.0,3072.0,153532.0,478.51,23.39,48.0,L-LC,11.33,185.0,0.13,1.11,23.25,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,5.47,-44.70,163.47,45.70,159833.0,-79040.0,97775.0,26.40,1.63,83.0,M-SC,22.75,204.0,-0.49,0.71,61.09,XY24,NTT,TRAVEL
7,ATULAUTO,-0.32,-11.54,64.55,45.57,112566.0,-22746.0,174386.0,844.00,3781.08,68.0,M-SC,4.42,248.0,-0.20,1.26,31.97,XY24,NTT,AUTO
11,BANDHANBNK,1.39,-17.85,134.23,92.42,307161.0,-49727.0,228832.0,400.00,96.12,71.0,H-SC,7.04,174.0,-0.16,1.66,32.25,XY24,NTT,BANKS
1,ABB,-0.87,6.15,37.14,45.57,103139.0,16085.0,277704.0,7934.00,-31.79,65.0,H-MC,4.27,125.0,0.16,2.01,23.32,AR,NTT,ELECTRICAL
67,SONACOMS,-0.07,-8.42,51.87,39.09,48055.0,-8514.0,92645.0,804.02,-27.49,63.0,M-MC,6.68,193.0,-0.18,0.67,31.06,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.19
1,25,44.67
2,50,76.76


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.43
MC    30.32
LC    23.25
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.95
X40      23.19
X40N     15.19
AR        8.97
XY25      8.81
XR        8.72
OX40N     7.30
SR        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.88
H-SC    22.70
X-LC    19.91
X-SC    11.87
M-SC    11.07
H-MC     4.93
M-MC     2.12
H-LC     1.16
L-LC     1.11
M-LC     1.07
L-SC     0.79
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.05,-14.09,52.42
FINANCE,13.49,-12.32,56.52
IT,11.43,-35.77,103.52
MISC,7.08,-33.25,87.75
ELECTRICAL,6.26,-8.66,48.31
PAINTS,5.09,-31.61,51.17
INSURANCE,4.89,-0.55,34.96
PHARMA,4.12,-3.56,36.16
AUTO,3.21,-18.08,58.91


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3254509.0,21
AR,1334056.0,10
X40,1299034.0,15
XR,1291537.0,12
X40N,1027548.0,10
OX40N,785739.0,10
XY25,400507.0,6
SR,299265.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3663215.0,24
M-SC,1392221.0,14
X-MC,1357423.0,15
X-SC,1168850.0,10
X-LC,1145404.0,12
H-MC,398833.0,3
L-SC,182099.0,2
M-LC,144794.0,1
M-MC,93011.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1232472.0      6
           AR         950316.0      5
           XR         832727.0      7
M-SC       XY24       797592.0      6
X-SC       XY24       607080.0      4
X-LC       X40        584044.0      6
X-MC       X40        539656.0      7
           X40N       415715.0      4
X-SC       X40N       386436.0      4
H-SC       OX40N      348435.0      4
M-SC       OX40N      344158.0      5
H-SC       SR         299265.0      2
X-LC       X40N       225397.0      2
H-MC       XY24       211380.0      1
X-LC       XY24       207369.0      2
X-MC       XY25       203436.0      2
           XY24       198616.0      2
H-MC       AR         187453.0      2
M-SC       XR         176245.0      2
X-SC       X40        175334.0      2
M-LC       XR         144794.0      1
X-LC       XY25       128594.0      2
L-SC       OX40N       93146.0      1
           XR          88953.0      1
M-SC       AR          74226.0      1
H-LC       AR          74006.0      1
L-MC       XR          48818.0      1
M-MC       AR          48055.0      1
           XY25        44956.0      1
L-LC       XY25        23521.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
